In [8]:
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from PIL import Image, ExifTags
import uuid
import cv2
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA

from scipy.spatial import distance 
from sklearn.metrics import pairwise_distances
from numba import jit, prange
from concurrent.futures import ProcessPoolExecutor

from scipy.spatial import distance
from sklearn.metrics import pairwise
import time
import sqlite3
import pickle

# Retrival Pipeline

In [2]:
%%time
'''
Load all image paths and get the total number of images,
we can't use tqdm here, because we have to determine the number of images first.
'''

def find_image_files(root_dir, extensions=('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif')):
    image_files = []
    
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith(extensions):
                image_files.append(os.path.join(subdir, file))
    return image_files


image_paths = find_image_files(r"C:\Users\timsa\Documents\DataTim\Weihnachten 2023\Bilder")
#df_image_paths = pd.DataFrame({'ID': range(1, len(image_paths) + 1), 'Path': image_paths})
#df_image_paths = pd.DataFrame(image_paths, columns=["Path"])
#df_image_paths.to_pickle('Path.pkl')

# Show first 10 paths
print(f"Number found images: {len(image_paths)}")
if len(image_paths) > 10:
    print("Some paths:")
    for pic_path in image_paths[:10]:
        print(pic_path)
else:
    print("Found paths:", image_paths)

Number found images: 11680
Some paths:
C:\Users\timsa\Documents\DataTim\Weihnachten 2023\Bilder\Screenshot_20230324_192205_WhatsApp.jpg
C:\Users\timsa\Documents\DataTim\Weihnachten 2023\Bilder\Screenshot_20230608_223514_Snapchat.jpg
C:\Users\timsa\Documents\DataTim\Weihnachten 2023\Bilder\Screenshot_20230709_152656_BeReal.jpg
C:\Users\timsa\Documents\DataTim\Weihnachten 2023\Bilder\Screenshot_20230715_030306_BeReal.jpg
C:\Users\timsa\Documents\DataTim\Weihnachten 2023\Bilder\Screenshot_20230720_212438_Snapchat.jpg
C:\Users\timsa\Documents\DataTim\Weihnachten 2023\Bilder\Screenshot_20230720_212749_Gallery.jpg
C:\Users\timsa\Documents\DataTim\Weihnachten 2023\Bilder\Screenshot_20230720_212756_Gallery.jpg
C:\Users\timsa\Documents\DataTim\Weihnachten 2023\Bilder\Screenshot_20230720_212830_Gallery.jpg
C:\Users\timsa\Documents\DataTim\Weihnachten 2023\Bilder\Screenshot_20230722_224505_Instagram.jpg
C:\Users\timsa\Documents\DataTim\Weihnachten 2023\Bilder\Screenshot_20230724_112125_BeReal.jpg

In [5]:
# Measurement functions (1st & 2nd)

def image_rgb_calculation(image):
    # OpenCV uses BGR color space, we have to convert it to the RGB color space
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Calculate the histogram
    hist = cv2.calcHist([rgb_image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    
    # Normalize the histogram so that histograms of different images (different sizes, resolutions) are comparable
    hist = cv2.normalize(hist, hist)
    hist = hist.flatten()
    return hist

def image_hsv_calculation(image):
    # OpenCV uses BGR color space, we have to convert it to the RGB color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Calculate the histogram (bin-sizes = 8, values from 0-255)
    hist = cv2.calcHist([hsv_image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    
    # Normalize the histogram so that histograms of different images (different sizes, resolutions) are comparable
    hist = cv2.normalize(hist, hist)
    hist = hist.flatten()
    return hist

# To be implemented:
def model_embeddings_calculation(model, image):
    pass

In [9]:
# Version 0

def image_batch_generator(image_files, batch_size, rezise_size, show_progress=True):
    total_batches = (len(image_files) + batch_size - 1) // batch_size
    progress_bar = tqdm(total=total_batches, desc="Processing images") if show_progress else None
    
    
    rgb_hists = []
    hsv_hists = []
    paths = []
    
    
    for index in range(0, len(image_files), batch_size):
        batch = image_files[index:index + batch_size]
        
        for i, path in enumerate(batch):
            image = cv2.imread(path)
            i = 1 + index + i
            if image is not None:
                try:
                    image = cv2.resize(image, rezise_size)
                    rgb_histogram = image_rgb_calculation(image)
                    hsv_histogram = image_hsv_calculation(image)
                    
                    rgb_hists.append((i, rgb_histogram))
                    hsv_hists.append((i, hsv_histogram))
                    paths.append((i, path))
                    
                except Exception as e:
                    print(f"Failed processing {path}: {e}")
                    continue
        
        #yield df, image_vectors
        if show_progress:
            progress_bar.update(1)

    if show_progress:
        progress_bar.close()
    
    return rgb_hists, hsv_hists, paths


batch_size = 100
desired_size = (60, 60)

rgb_hists, hsv_hists, paths = image_batch_generator(image_paths, batch_size, desired_size, show_progress=True)

df_rgb = pd.DataFrame(rgb_hists, columns=["ID", "Histogram"])
df_paths = pd.DataFrame(rgb_hists, columns=["ID", "Path"])
df_rgb.to_pickle('RGB_Hist.pkl')
df_paths.to_pickle('Path.pkl')
df_paths

Processing images:   0%|          | 0/1 [00:00<?, ?it/s]

,ID,Path
0,1,"[0.336425, 0.019351881, 0.0, 0.0, 0.0, 0.0, 0...."
1,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,3,"[0.9998705, 0.00057862874, 0.0, 0.0, 0.0, 0.0,..."
3,5,"[0.25492105, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,6,"[0.7242774, 0.013039364, 0.0, 0.0, 0.0, 0.0, 0..."
5,7,"[0.7743936, 0.0060108174, 0.0, 0.0, 0.0, 0.0, ..."
6,8,"[0.035986535, 0.0071973074, 0.0, 0.0, 0.0, 0.0..."
7,9,"[0.052002728, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
8,10,"[0.029992608, 0.00085693167, 0.0, 0.0, 0.0, 0...."
9,11,"[0.40314555, 0.00842813, 0.0, 0.0, 0.0, 0.0, 0..."


In [6]:
# Different Version 1 (less data)


def extract_image_details(image_id, path, resize_size):
    try:
        image = cv2.imread(path)
        if image is not None:
            image = cv2.resize(image, resize_size)
            rgb_histogram = image_rgb_calculation(image)
            hsv_histogram = image_hsv_calculation(image)
            return {
                'ID': image_id,
                'Path': path,
                'RGB_Histogram': rgb_histogram,
                'HSV_Histogram': hsv_histogram
            }
        
        else:
            print(f"Image at path {path} is None.")
            return None
        
    except Exception as e:
        print(f"Failed processing {path}: {e}")
        return None


def image_batch_generator(image_files, batch_size, resize_size, show_progress=True):
    total_batches = (len(image_files) + batch_size - 1) // batch_size
    progress_bar = tqdm(total=total_batches, desc="Processing images") if show_progress else None

    for index in range(0, len(image_files), batch_size):
        batch = image_files[index:index + batch_size]
        details_list = [extract_image_details(index + i + 1, path, resize_size) for i, path in enumerate(batch)]
        details_list = [features for features in details_list if features is not None]
        df = pd.DataFrame(details_list)
        yield df
        if show_progress:
            progress_bar.update(1)

    if show_progress:
        progress_bar.close()

        
        
# Beispielaufruf
batch_size = 100
desired_size = (60, 60)


rgb_hists, hsv_hists, paths = [], [], []
for df in image_batch_generator(image_paths, batch_size, desired_size, show_progress=True):
    paths.extend(df[['ID', 'Path']].values.tolist())
    rgb_hists.extend(df[['ID', 'RGB_Histogram']].values.tolist())
    hsv_hists.extend(df[['ID', 'HSV_Histogram']].values.tolist())
    

df_rgb = pd.DataFrame(rgb_hists, columns=["ID", "Histogram"])
df_hsv = pd.DataFrame(hsv_hists, columns=["ID", "Histogram"])
df_paths = pd.DataFrame(paths, columns=["ID", "Path"])

df_rgb.to_pickle('RGB_Hist.pkl')
df_hsv.to_pickle('HSV_Hist.pkl')
df_paths.to_pickle('Path.pkl')

print(df_rgb.head())
print(df_hsv.head())
print(df_paths.head())
 

Processing images:   0%|          | 0/117 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
# Different Version 2 (more data)


def extract_image_details(image_id, path, resize_size):
    try:
        image = cv2.imread(path)
        if image is not None:
            image = cv2.resize(image, resize_size)
            rgb_histogram = image_rgb_calculation(image)
            hsv_histogram = image_hsv_calculation(image)
            
            
            ### several more informations ###
            
            # Resize image
            resized_image = cv2.resize(image, resize_size)
            
            # Flatten image
            resized_image_vector = resized_image.flatten()
            
            # Convert to RGB
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Calculate average color and brightness
            avg_color = np.mean(image_rgb, axis=(0, 1)).tolist()
            avg_brightness = np.mean(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))

            # Convert to HSV and calculate average HSV
            image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            avg_hsv = np.mean(image_hsv, axis=(0, 1)).tolist()

            # File details
            file_size = os.path.getsize(path)
            file_type = os.path.splitext(path)[1]

            with Image.open(path) as img:
                # Resolution and DPI
                resolution = img.size  # (width, height)
                dpi = img.info.get('dpi', (0, 0))
                
                # Extract metadata
                try:
                    exif_data = img._getexif()
                    metadata = {ExifTags.TAGS.get(k, k): v for k, v in exif_data.items()} if exif_data else {}
                except AttributeError:
                    metadata = {}
            
            
            #################################
            
            
            return {
                'ID': image_id,
                'Path': path,
                'RGB_Histogram': rgb_histogram,
                'HSV_Histogram': hsv_histogram,
                'Average_Color': avg_color,
                'Brightness': avg_brightness,
                'Average_HSV': avg_hsv,
                'Resolution': resolution,
                'DPI': dpi,
                'File_Size': file_size,
                'File_Type': file_type,
                'Metadata': metadata,
                'Resized_Image_Vector': resized_image_vector
            }
        
        else:
            print(f"Image at path {path} is None.")
            return None
        
    except Exception as e:
        print(f"Failed processing {path}: {e}")
        return None

    
def load_checkpoint():
    checkpoints = sorted([f for f in os.listdir() if f.startswith('checkpoint_') and f.endswith('.pkl')])
    
    if not checkpoints:
        return 0, [], [], [], []
    
    latest_checkpoint = checkpoints[-1]
    
    with open(latest_checkpoint, 'rb') as f:
        batch_index, paths, rgb_hists, hsv_hists, other_data = pickle.load(f)

    return batch_index, paths, rgb_hists, hsv_hists, other_data
    
    

def image_batch_generator(image_files, batch_size, resize_size, start_index = 0, show_progress=True):
    total_batches = (len(image_files) + batch_size - 1) // batch_size
    progress_bar = tqdm(total=total_batches, desc="Processing images") if show_progress else None

    for index in range(start_index, len(image_files), batch_size):
        batch = image_files[index:index + batch_size]
        details_list = [extract_image_details(index + i + 1, path, resize_size) for i, path in enumerate(batch)]
        details_list = [features for features in details_list if features is not None]
        df = pd.DataFrame(details_list)
        yield df, index + batch_size
        if show_progress:
            progress_bar.update(1)

    if show_progress:
        progress_bar.close()

        
        
# Testing: Example
batch_size = 100
desired_size = (60, 60)


# Load checkpoint
start_index, paths, rgb_hists, hsv_hists, other_data = load_checkpoint()


for df, batch_index in image_batch_generator(image_paths, batch_size, desired_size, start_index=start_index, show_progress=True):
    paths.extend(df[['ID', 'Path']].values.tolist())
    rgb_hists.extend(df[['ID', 'RGB_Histogram']].values.tolist())
    hsv_hists.extend(df[['ID', 'HSV_Histogram']].values.tolist())
    other_data.extend(df.drop(columns=['Path', 'RGB_Histogram', 'HSV_Histogram']).values.tolist())
    
    # Save checkpoint
    with open(f'checkpoint_{batch_index}.pkl', 'wb') as f:
        pickle.dump((batch_index, paths, rgb_hists, hsv_hists, other_data), f)
    

# Save results
df_rgb = pd.DataFrame(rgb_hists, columns=["ID", "Histogram"])
df_hsv = pd.DataFrame(hsv_hists, columns=["ID", "Histogram"])
df_paths = pd.DataFrame(paths, columns=["ID", "Path"])
df_other_data = pd.DataFrame(other_data, columns=["ID", "Average_Color",
                                                 "Brightness", "Average_HSV",
                                                 "Resolution","DPI",
                                                 "File_Size", "File_Type",
                                                 "Metadata", "Resized_Image_Vector"
                                                ])
 
df_rgb.to_pickle('RGB_Hist.pkl')
df_hsv.to_pickle('HSV_Hist.pkl')
df_paths.to_pickle('Path.pkl')
df_other_data.to_pickle('Other_data.pkl')

Processing images:   0%|          | 0/117 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [14]:
start_index, paths, rgb_hists, hsv_hists, other_data = load_checkpoint()

In [17]:
other_data

[[1,
  [46.54611111111111, 40.53472222222222, 36.04611111111111],
  41.81361111111111,
  [63.764722222222225, 103.79638888888888, 50.02194444444444],
  (1080, 2340),
  (72, 72),
  1142547,
  '.jpg',
  {'ImageWidth': 1080,
   'ImageLength': 2340,
   'ExifOffset': 112,
   'Software': 'Android TP1A.220624.014.S916BXXS1AWBM',
   'Orientation': 0,
   'LightSource': 0,
   'SubsecTimeOriginal': '601',
   'DateTimeOriginal': '2023:03:24 19:22:05',
   'OffsetTimeOriginal': '+01:00'},
  array([16, 12,  7, ...,  0,  0,  0], dtype=uint8)],
 [2,
  [104.95305555555555, 87.01444444444445, 58.57555555555555],
  89.12861111111111,
  [25.288333333333334, 125.20583333333333, 106.80222222222223],
  (1080, 1817),
  (72, 72),
  409177,
  '.jpg',
  {'ImageWidth': 1080,
   'ImageLength': 2340,
   'ExifOffset': 144,
   'Software': 'Android TP1A.220624.014.S916BXXS1AWD1',
   'Orientation': 1,
   'DateTime': '2023:06:08 22:35:14',
   'OffsetTime': '+02:00',
   'SubsecTimeOriginal': '439',
   'OffsetTimeOriginal'

In [5]:
len(df_rgb)

16

In [6]:
print(df_rgb.tail())
print(df_hsv.tail())
print(df_paths.tail())
print(df_other_data.tail())

    ID                                          Histogram
11  13  [0.20665304, 0.016907977, 0.0, 0.0, 0.0, 0.0, ...
12  14  [0.2258655, 0.015666971, 0.001305581, 0.0, 0.0...
13  15  [0.10874878, 0.07299575, 0.0, 0.0, 0.0, 0.0, 0...
14  16  [0.2071246, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
15  17  [0.4422131, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
    ID                                          Histogram
11  13  [0.011075616, 0.0, 0.005537808, 0.044302464, 0...
12  14  [0.008913387, 0.002546682, 0.002546682, 0.0050...
13  15  [0.01610175, 0.050094333, 0.028625334, 0.01789...
14  16  [0.034017637, 0.014323216, 0.030436834, 0.0429...
15  17  [0.018316956, 0.0033303557, 0.0016651779, 0.00...
    ID                                               Path
11  13  C:\Users\timsa\Desktop\sample_pictures\folder_...
12  14  C:\Users\timsa\Desktop\sample_pictures\folder_...
13  15  C:\Users\timsa\Desktop\sample_pictures\folder_...
14  16  C:\Users\timsa\Desktop\sample_pictures\folder_...
15  17  C:\Use

In [7]:
# Create database

# Check if the folder 'databases' does not exist / create it
if not os.path.exists('database'):
    os.makedirs('database')
    
conn = sqlite3.connect("database/bd_database.db")
curs = conn.cursor()

curs.execute("""CREATE TABLE IF NOT EXISTS image_paths 
                (ID INTEGER PRIMARY KEY,
                Path text);""")
conn.commit()

In [8]:
# Save data to database
path_pickle_df = pd.read_pickle("Path.pkl")

for file_path in path_pickle_df['Path']:
    curs.execute('''INSERT OR IGNORE INTO image_paths (Path) VALUES (?);''', (file_path,))
    # print(f"Inserted path: {file_path}")
    
conn.commit()

In [9]:
curs.execute("""SELECT *
                FROM image_paths;""")
results = curs.fetchall()

df = pd.DataFrame(results, columns=['ID', 'Path'])

#conn.close()

df

,ID,Path
0,1,C:\Users\timsa\Desktop\sample_pictures\0031.png
1,2,C:\Users\timsa\Desktop\sample_pictures\adam-bi...
2,3,C:\Users\timsa\Desktop\sample_pictures\adrian-...
3,4,C:\Users\timsa\Desktop\sample_pictures\folder_...
4,5,C:\Users\timsa\Desktop\sample_pictures\folder_...
5,6,C:\Users\timsa\Desktop\sample_pictures\folder_...
6,7,C:\Users\timsa\Desktop\sample_pictures\folder_...
7,8,C:\Users\timsa\Desktop\sample_pictures\folder_...
8,9,C:\Users\timsa\Desktop\sample_pictures\folder_...
9,10,C:\Users\timsa\Desktop\sample_pictures\folder_...


In [11]:
# Similarity functions

def euclidean_distance():
    pass

def manhattan_distance():
    pass

def cosine_similarity():
    pass

def jaccard_similarity():
    pass

def hamming_distance():
    pass


# ___________________________________________________

In [70]:
# Parts for measurment - dimensionality reduction
# needs 1-D-vector


def extract_image_details(image_path):
    try:
        with Image.open(image_path) as img:
            img = img.convert('RGB')  # Convert to RGB

            # Resize image
            img_cv = cv2.imread(image_path)
            resized = cv2.resize(img_cv, desired_size)
            img_as_1d = np.array(resized).flatten() #this could be parallelized

            # UUID
            unique_id = str(uuid.uuid4())

        return {
            'ID': unique_id,
            'File_Path': image_path,
            'Resized_Image_Vector': img_as_1d
        }
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

In [71]:
def process_large_image_dataset(generator, total_images, batch_size, n_components):
    """
    Processes a large image dataset using IncrementalPCA in batches.

    Parameters:
    generator (generator): Generator yielding batches of image data.
    total_images (int): Total number of images in the dataset.
    batch_size (int): Number of images to process in each batch.
    n_components (int): Number of principal components to keep.

    Returns:
    IncrementalPCA: The fitted IncrementalPCA model.
    """
    ipca = IncrementalPCA(n_components=n_components)
    
    for _ in range(0, total_images, batch_size):
        batch_df, image_vectors = next(generator)
        ipca.partial_fit(image_vectors)
    
    return ipca


def transform_image_features(batch_df, image_vectors, ipca):
    """
    Transforms image features using PCA.

    Parameters:
    batch_df (pd.DataFrame): The DataFrame containing batch image details.
    image_vectors (np.array): The array of image vectors.
    ipca (IncrementalPCA): The pre-fitted IncrementalPCA.

    Returns:
    pd.DataFrame: The updated DataFrame with PCA vectors added.
    """
    pca_vectors = ipca.transform(image_vectors)
    batch_df['PCA_Vectors'] = list(pca_vectors)
    return batch_df


In [72]:
def image_batch_generator(image_files, batch_size, show_progress=True):
    total_batches = (len(image_files) + batch_size - 1) // batch_size
    progress_bar = tqdm(total=total_batches, desc="Processing images") if show_progress else None

    for index in range(0, len(image_files), batch_size):
        batch = image_files[index:index + batch_size]
        details_list = [extract_image_details(image) for image in batch]
        details_list = [features for features in details_list if features is not None]
        image_vectors = np.array([features['Resized_Image_Vector'] for features in details_list])
        df = pd.DataFrame(details_list)
        yield df, image_vectors
        if show_progress:
            progress_bar.update(1)

    if show_progress:
        progress_bar.close()

In [74]:
# Execution of code

total_images = len(image_paths)
batch_size = 100
desired_size = (60, 60)   # size which the images will be resized to
n_components = 10 # the amount of features our iPCA will keep for every image

generator = image_batch_generator(image_paths, batch_size, show_progress=True)   # gives back batch_df + image as 1d
ipca = process_large_image_dataset(generator, total_images, batch_size, n_components) # needed to fit the iPCA 

# After fitting IPCA, transform all data
for batch_df, image_vectors in image_batch_generator(image_paths, batch_size, show_progress=False):
    transformed_batch_df = transform_image_features(batch_df, image_vectors, ipca)

    
    transformed_batch_df.head()   


Processing images:   0%|          | 0/1 [00:00<?, ?it/s]

Error processing C:\Users\timsa\Desktop\sample_pictures\test_2.bmp: cannot identify image file 'C:\\Users\\timsa\\Desktop\\sample_pictures\\test_2.bmp'
Error processing C:\Users\timsa\Desktop\sample_pictures\test_2.bmp: cannot identify image file 'C:\\Users\\timsa\\Desktop\\sample_pictures\\test_2.bmp'


In [75]:
transformed_batch_df.head()

,ID,File_Path,Resized_Image_Vector,PCA_Vectors
0,34b0bb62-1f7b-45a8-8fc4-eaedddb5757f,C:\Users\timsa\Desktop\sample_pictures\0031.png,"[195, 187, 177, 188, 167, 162, 144, 123, 139, ...","[496.6541985933603, -888.0665161932402, -3756...."
1,8c0dfbb5-bba4-409d-8b57-849ce8bde435,C:\Users\timsa\Desktop\sample_pictures\adam-bi...,"[222, 212, 208, 224, 213, 207, 225, 216, 209, ...","[12632.571816986021, -259.16952814348303, 2069..."
2,e07de2bc-620f-4a07-b2b9-653b6c86f0b6,C:\Users\timsa\Desktop\sample_pictures\adrian-...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-11009.597883374357, -501.0693401087662, -502..."
3,d332eb0a-e650-4c85-a5ae-cec91dfc3c8c,C:\Users\timsa\Desktop\sample_pictures\folder_...,"[98, 98, 94, 41, 48, 38, 20, 32, 32, 14, 26, 2...","[-3963.8594334778113, 2623.8053094494276, 1098..."
4,47e15f53-ee2d-42ac-81c7-a4e302982c86,C:\Users\timsa\Desktop\sample_pictures\folder_...,"[55, 51, 61, 36, 27, 30, 110, 51, 32, 161, 113...","[-4741.879883035555, -2071.263536722551, 1369...."


In [76]:
transformed_batch_df["PCA_Vectors"].describe()

count                                                    16
unique                                                   16
top       [496.6541985933603, -888.0665161932402, -3756....
freq                                                      1
Name: PCA_Vectors, dtype: object

In [77]:
transformed_batch_df["PCA_Vectors"][1]

array([12632.57181699,  -259.16952814,  2069.72584842,   444.01229774,
         481.85474691,   -62.15674047,   918.43916599,   427.64807317,
         293.81656367,   -74.17618535])